In [1]:
import numpy as np
import os
from glob import glob
import tensorflow as tf
import tensorflow.contrib.eager as tfe
import cv2
import matplotlib.pyplot as plt
from natsort import natsorted

/misc/me/pratikm/virtualEnv/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


## Global variables

In [2]:
DIR_DATA = '../test'
DIR_INPUT = os.path.join(DIR_DATA, 'input')
DIR_OUTPUT = os.path.join(DIR_DATA, 'output')

IMG_OUT_H, IMG_OUT_W = 128, 128

DIR_TFRECORDS = 'data_tfrecords/test'
if not os.path.exists(DIR_TFRECORDS):
    os.makedirs(DIR_TFRECORDS)

## Process data

In [3]:
def get_processed_data(img, mask, idx_class):
    # resize all images to fixed size
    img = cv2.resize(img, (IMG_OUT_W, IMG_OUT_W))
    mask = cv2.resize(mask, (IMG_OUT_W, IMG_OUT_W))  
    
    idx_class = np.array(idx_class, dtype=np.float32)
    
    return img, mask, idx_class

## Write data to tfrecord format

In [4]:
# conversion functions (data to feature data types)
def _bytes_feature(value):
    return tf.train.Feature(bytes_list=tf.train.BytesList(value=[value]))

def write_example_to_tfrecord(img, mask, idx_class, writer):
    # get processed data
    img, mask, idx_class = get_processed_data(img, mask, idx_class)
    
    # create example from this data
    example = tf.train.Example(
        features=tf.train.Features(
            feature={
                'img': _bytes_feature(img.tostring()),
                'mask': _bytes_feature(mask.tostring()),
                'idx_class': _bytes_feature(idx_class.tostring())
            }
        )
    )

    writer.write(example.SerializeToString())


def write_data_to_tfrecord(folder_name, idx_class):
    # read filenames
    filenames = sorted(os.listdir(os.path.join(DIR_INPUT, folder_name)))
    
    # write data into single TFRecord files
    idx_data = 0
    
    # new TFRecord file
    filename_tfrecord = os.path.join(DIR_TFRECORDS, folder_name + '.tfrecords')
    with tf.python_io.TFRecordWriter(filename_tfrecord) as writer:
        for filename in filenames:                
            # read input
            path = os.path.join(os.path.join(DIR_INPUT, folder_name), filename)
            img = cv2.imread(path)
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            
            # read output
            path = os.path.join(os.path.join(DIR_OUTPUT, folder_name), filename)
            mask = cv2.imread(path, cv2.IMREAD_GRAYSCALE)
            
            # write to file
            write_example_to_tfrecord(img, mask, idx_class, writer)

In [5]:
if __name__ == '__main__':
    # read all folder_names
    folder_names = sorted(os.listdir(DIR_INPUT), key=lambda s: s.lower())
    
    # write each folder into tfrecords
    for idx_class, folder_name in enumerate(folder_names):
        print('Reading from', folder_name)
        write_data_to_tfrecord(folder_name, idx_class)

Reading from accordion
Reading from airplanes
Reading from anchor
Reading from ant
Reading from barrel
Reading from bass
Reading from beaver
Reading from binocular
Reading from bonsai
Reading from brain
Reading from brontosaurus
Reading from buddha
Reading from butterfly
Reading from camera
Reading from cannon
Reading from car_side
Reading from ceiling_fan
Reading from cellphone
Reading from chair
Reading from chandelier
Reading from cougar_body
Reading from cougar_face
Reading from crab
Reading from crayfish
Reading from crocodile
Reading from crocodile_head
Reading from cup
Reading from dalmatian
Reading from dollar_bill
Reading from dolphin
Reading from dragonfly
Reading from electric_guitar
Reading from elephant
Reading from emu
Reading from euphonium
Reading from ewer
Reading from Faces
Reading from Faces_easy
Reading from ferry
Reading from flamingo
Reading from flamingo_head
Reading from garfield
Reading from gerenuk
Reading from gramophone
Reading from grand_piano
Reading from 